In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,trainer
import torch

/home/adminad/zhangdw/workspace/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path='/data/download-model/Qwen3-0.6B'

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForCausalLM.from_pretrained(model_path)

In [4]:
sentence='1*1=1, 2*2=4, 3*3=9, 2*3*2*3='

In [6]:
inputs=tokenizer(sentence,return_tensors="pt")

In [7]:
inputs.input_ids.shape

torch.Size([1, 29])

In [9]:
tokenizer.batch_decode(inputs.input_ids)

['1*1=1, 2*2=4, 3*3=9, 2*3*2*3=']

In [246]:
inputs['labels']=inputs['input_ids']

In [247]:
outputs=model(**inputs)

In [248]:
outputs.logits.shape

torch.Size([1, 29, 151936])

In [249]:
outputs

CausalLMOutputWithPast(loss=tensor(1.1780, grad_fn=<NllLossBackward0>), logits=tensor([[[  3.6384,   2.8918,   2.5816,  ...,  -4.1986,  -4.2038,  -4.2262],
         [  1.3595,  -4.9326,   1.0736,  ...,  -9.8884,  -9.9113, -10.0890],
         [  8.2963,   5.3321,   5.6211,  ...,  -7.2254,  -7.1978,  -7.2971],
         ...,
         [ 14.3291,   8.1484,  14.0710,  ...,   3.7847,   3.8039,   3.9626],
         [  7.5321,   7.5086,   7.4076,  ...,  -3.2272,  -3.1894,  -3.1203],
         [  9.2310,   5.6040,   7.7539,  ...,   2.0228,   2.0447,   2.1962]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7fe9875eb910>, hidden_states=None, attentions=None)

In [255]:
outputs.logits.argmax(dim=2).shape

torch.Size([1, 29])

In [251]:
tokenizer.batch_decode(outputs.logits.argmax(dim=2))

['.101, 1*2=4, 3*3=9, 4*5=6=3=3']

In [252]:
tokenizer.batch_decode(model.generate(**inputs))

['1*1=1, 2*2=4, 3*3=9, 2*3*2*3=36, 36/36=1, 1+1=2, ']

In [55]:
tokenizer.pad_token

'<|endoftext|>'

In [122]:
# 输入提示文本
prompt = "<|endoftext|>Once upon a time "
target_text = "<|endoftext|>Once upon a time ?"

# 编码输入和目标文本
inputs = tokenizer(prompt, return_tensors="pt")

targets = tokenizer(target_text, return_tensors="pt")

# labels=torch.cat([torch.tensor([[-100]*5]),targets['input_ids']],dim=-1)

inputs['labels']=targets['input_ids']

# # 验证形状一致性
print(f"input_ids shape: {inputs.input_ids.shape}")  
print(f"labels shape: {inputs.labels.shape}")

input_ids shape: torch.Size([1, 6])
labels shape: torch.Size([1, 6])


In [123]:
tokenizer.batch_decode(model.generate(**inputs))

['<|endoftext|>Once upon a time  (also known as "the story of the first time") was a 2011 comedy']

In [124]:
outputs=model(**inputs)

In [125]:
tokenizer.batch_decode(outputs.logits.argmax(dim=2))

['QuestionOnce a time,1']

In [126]:
outputs.loss

tensor(6.5680, grad_fn=<NllLossBackward0>)

In [127]:
outputs.loss.backward()

In [109]:
model.train()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe